In [2]:
import pandas as pd

deposits = pd.read_csv("deposits.csv")
withdrawals = pd.read_csv("withdrawals.csv")
gameplay = pd.read_csv("gameplay.csv")


In [3]:
import pandas as pd
import numpy as np

# Convert 'Datetime' to datetime
deposits['Datetime'] = pd.to_datetime(deposits['Datetime'], errors='coerce')
withdrawals['Datetime'] = pd.to_datetime(withdrawals['Datetime'], errors='coerce')
gameplay['Datetime'] = pd.to_datetime(gameplay['Datetime'], errors='coerce')

# Create date and slot columns
for df in [deposits, withdrawals, gameplay]:
    df['date'] = df['Datetime'].dt.date
    df['slot'] = df['Datetime'].apply(lambda x: 'S1' if x.hour < 12 else 'S2')


In [4]:
# Try printing first few rows
print(deposits.head())
print(withdrawals.head())
print(gameplay.head())


   User Id            Datetime       Date         Time  Amount Slot  \
0    357.0 2022-01-10 00:03:00  1/10/2022  12:03:00 AM  2000.0   S1   
1    776.0 2022-01-10 00:03:00  1/10/2022  12:03:00 AM  2500.0   S1   
2    492.0 2022-01-10 00:06:00  1/10/2022  12:03:00 AM  5000.0   S1   
3    803.0 2022-01-10 00:07:00  1/10/2022  12:06:00 AM  5000.0   S1   
4    875.0 2022-01-10 00:09:00  1/10/2022  12:07:00 AM  1500.0   S1   

         date slot  
0  2022-01-10   S1  
1  2022-01-10   S1  
2  2022-01-10   S1  
3  2022-01-10   S1  
4  2022-01-10   S1  
   User Id            Datetime       Date         Time  Amount Slot  \
0      190 2022-01-10 00:03:00  1/10/2022  12:03:00 PM    5872   S1   
1      159 2022-01-10 00:16:00  1/10/2022  12:03:00 PM    9540   S1   
2      164 2022-01-10 00:24:00  1/10/2022  12:16:00 PM     815   S1   
3      946 2022-01-10 00:29:00  1/10/2022  12:24:00 PM   23000   S1   
4      763 2022-01-10 00:40:00  1/10/2022  12:29:00 PM    9473   S1   

         date slot  

In [5]:
# Deposits aggregation
deposit_agg = deposits.groupby(['User Id', 'date', 'slot']).agg(
    deposit_amount_sum=('Amount', 'sum'),
    deposit_count=('Amount', 'count')
).reset_index()

# Withdrawals aggregation
withdraw_agg = withdrawals.groupby(['User Id', 'date', 'slot']).agg(
    withdraw_amount_sum=('Amount', 'sum'),
    withdraw_count=('Amount', 'count')
).reset_index()

# Gameplay aggregation (pehle User ID rename)
gameplay.rename(columns={'User ID': 'User Id'}, inplace=True)

gameplay_agg = gameplay.groupby(['User Id', 'date', 'slot']).agg(
    games_played=('Games Played', 'sum')
).reset_index()


In [6]:
print(deposit_agg.head())


   User Id        date slot  deposit_amount_sum  deposit_count
0      2.0  2022-01-10   S2             40000.0              1
1      2.0  2022-02-10   S2              5000.0              1
2      2.0  2022-06-10   S1             40000.0              1
3      2.0  2022-08-10   S1             45000.0              2
4      2.0  2022-08-10   S2              5000.0              1


In [7]:
print(withdraw_agg.head())


   User Id        date slot  withdraw_amount_sum  withdraw_count
0        2  2022-01-10   S2                55000               1
1        2  2022-03-10   S1               117349               1
2        2  2022-04-10   S2                19000               1
3        2  2022-05-10   S1                36000               1
4        2  2022-08-10   S2                13000               1


In [8]:
print(gameplay_agg.head())


   User Id        date slot  games_played
0      0.0  2022-04-10   S2           1.0
1      0.0  2022-05-10   S2           1.0
2      0.0  2022-06-10   S1           1.0
3      0.0  2022-10-10   S1           1.0
4      0.0  2022-10-10   S2           1.0


In [9]:
# Merge deposits and withdrawals
df = pd.merge(deposit_agg, withdraw_agg, on=['User Id', 'date', 'slot'], how='outer')

# Merge gameplay
df = pd.merge(df, gameplay_agg, on=['User Id', 'date', 'slot'], how='outer')

# Fill NaNs with 0
df.fillna(0, inplace=True)


In [14]:
print("Step 3 - merged df shape:", df.shape)
print(df.head())
print(df.columns)


Step 3 - merged df shape: (15161, 9)
   User Id        date slot  deposit_amount_sum  deposit_count  \
0      0.0  2022-04-10   S2                 0.0            0.0   
1      0.0  2022-05-10   S2                 0.0            0.0   
2      0.0  2022-06-10   S1                 0.0            0.0   
3      0.0  2022-10-10   S1                 0.0            0.0   
4      0.0  2022-10-10   S2                 0.0            0.0   

   withdraw_amount_sum  withdraw_count  games_played  loyalty_points  
0                  0.0             0.0           1.0             0.2  
1                  0.0             0.0           1.0             0.2  
2                  0.0             0.0           1.0             0.2  
3                  0.0             0.0           1.0             0.2  
4                  0.0             0.0           1.0             0.2  
Index(['User Id', 'date', 'slot', 'deposit_amount_sum', 'deposit_count',
       'withdraw_amount_sum', 'withdraw_count', 'games_played',
   

In [10]:
import numpy as np

df['loyalty_points'] = (
    0.01 * df['deposit_amount_sum'] +                    # 1% of total deposited amount
    0.005 * df['withdraw_amount_sum'] +                  # 0.5% of total withdrawn amount
    0.001 * np.maximum(df['deposit_count'] - df['withdraw_count'], 0) +  # small bonus for net deposit transactions
    0.2 * df['games_played']                             # 0.2 points per game played
)


In [15]:
print("Step 4 - sample loyalty points:")
print(df[['User Id', 'date', 'slot', 'loyalty_points']].head(10))
print("Min loyalty points:", df['loyalty_points'].min())
print("Max loyalty points:", df['loyalty_points'].max())


Step 4 - sample loyalty points:
   User Id        date slot  loyalty_points
0      0.0  2022-04-10   S2             0.2
1      0.0  2022-05-10   S2             0.2
2      0.0  2022-06-10   S1             0.2
3      0.0  2022-10-10   S1             0.2
4      0.0  2022-10-10   S2             0.2
5      0.0  2022-11-10   S2             0.4
6      1.0  2022-01-10   S1             0.2
7      1.0  2022-07-10   S1             0.2
8      2.0  2022-01-10   S1             0.4
9      2.0  2022-01-10   S2           675.2
Min loyalty points: 0.2
Max loyalty points: 3822.64


In [11]:
from datetime import date

# Filter for October 2022 data
october_df = df[(df['date'] >= date(2022, 10, 1)) & (df['date'] <= date(2022, 10, 31))]


In [16]:
print("Step 5 - October data shape:", october_df.shape)
print(october_df['date'].min(), october_df['date'].max())


Step 5 - October data shape: (1262, 9)
2022-10-10 2022-10-10


In [12]:
# Aggregate loyalty points by User Id for the month of October
user_points = october_df.groupby('User Id')['loyalty_points'].sum().reset_index()

# Sort descending by loyalty points and get top 50
top_50_players = user_points.sort_values(by='loyalty_points', ascending=False).head(50)


In [17]:
print("Step 6 - Top 50 players:")
print(top_50_players.head())
print("Number of players:", len(top_50_players))
print("Max loyalty points in top 50:", top_50_players['loyalty_points'].max())
print("Min loyalty points in top 50:", top_50_players['loyalty_points'].min())


Step 6 - Top 50 players:
     User Id  loyalty_points        bonus
469    634.0        5164.630  1548.318678
445    601.0        2200.001   659.544370
324    438.0        2000.001   599.585818
530    714.0        1380.204   413.775166
255    344.0        1350.000   404.720225
Number of players: 50
Max loyalty points in top 50: 5164.63
Min loyalty points in top 50: 238.89400000000003


In [13]:
# Example: proportional bonus based on loyalty points share
total_points = top_50_players['loyalty_points'].sum()
top_50_players['bonus'] = top_50_players['loyalty_points'] / total_points * 10000  # e.g., total bonus pool = 10,000


In [18]:
print("Step 7 - bonus distribution sample:")
print(top_50_players[['User Id', 'loyalty_points', 'bonus']].head())

print("Total bonus allocated:", top_50_players['bonus'].sum())


Step 7 - bonus distribution sample:
     User Id  loyalty_points        bonus
469    634.0        5164.630  1548.318678
445    601.0        2200.001   659.544370
324    438.0        2000.001   599.585818
530    714.0        1380.204   413.775166
255    344.0        1350.000   404.720225
Total bonus allocated: 10000.0


In [19]:
print(top_50_players.head(10))  # Show top 10 users with loyalty points and bonus
print("Total bonus distributed:", top_50_players['bonus'].sum())
print("Number of players in leaderboard:", len(top_50_players))


     User Id  loyalty_points        bonus
469    634.0        5164.630  1548.318678
445    601.0        2200.001   659.544370
324    438.0        2000.001   599.585818
530    714.0        1380.204   413.775166
255    344.0        1350.000   404.720225
421    569.0        1116.602   334.749195
400    540.0        1114.201   334.029392
666    890.0        1001.001   300.092852
687    920.0         960.952   288.086452
419    566.0         951.003   285.103813
Total bonus distributed: 10000.0
Number of players in leaderboard: 50


In [20]:
total_bonus_pool = 10000  # or whatever you set
allocated_bonus_sum = top_50_players['bonus'].sum()
assert abs(allocated_bonus_sum - total_bonus_pool) < 1e-6, "Bonus distribution mismatch!"


In [21]:
print(top_50_players.isnull().sum())
print((top_50_players['loyalty_points'] <= 0).sum())


User Id           0
loyalty_points    0
bonus             0
dtype: int64
0


In [22]:
top_50_players.to_csv('top_50_loyalty_leaderboard.csv', index=False)
